In [29]:
from collections import Counter
import re

import nltk
import numpy as np
import pandas as pd

In [50]:
class TrigramTCG():
    def _convert_text(self):
        self._trigrams = dict()

        for s in self._X.values:
            s = ['<<START2>>', '<<START1>>'] + s + ['<<END1>>', '<<END2>>']
            for a, b, c in zip(s, s[1:], s[2:]):
                if a in self._trigrams:
                    if b in self._trigrams[a]:
                        self._trigrams[a][b][c] += 1
                    else:
                        self._trigrams[a][b] = Counter({c: 1})
                else:
                    self._trigrams[a] = {b: Counter({c: 1})}

        for k, v in self._trigrams.items():
            for k2, v2 in v.items():
                s = sum(v2.values())
                for k3, v3 in v2.items():
                    self._trigrams[k][k2][k3] /= s
                    
    def _format_comment(self, comment):
        comment = re.sub(r" (?=[.\"'?!,)-])", "", comment)
        comment = re.sub(r"(?<=[']) ", "", comment)

        return comment

    def process_comments(self, path='./toxic/train.csv', pos_flags=['toxic'], neg_flags=['identity_hate']):
        '''
        pos_flags: Only use comments where pos_flags are 1
        neg_flags: Only use comments where neg_flags are 0
        Flags can be 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'
        '''
        df = pd.read_csv(path)
        self._X = df[np.all(df[pos_flags]==1, 1)&np.all(df[neg_flags]==0, 1)].comment_text
        self._X = self._X.apply(lambda x: nltk.tokenize.ToktokTokenizer().tokenize(x.lower()))

        self._convert_text()
                    
    def generate_comment(self, random_start=False):
        comment = []
        while comment == []:
            if not random_start:
                a, b = '<<START2>>', '<<START1>>'
            else:
                a, b = "", "<<END1>>"
                while b == "<<END1>>":
                    a = np.random.choice(list(self._trigrams.keys()))
                    b = np.random.choice(list(self._trigrams[a].keys()))

            c = np.random.choice(list(self._trigrams[a][b].keys()), p=list(self._trigrams[a][b].values()))

            while c != '<<END1>>':
                comment.append(c)
                a = b
                b = c
                c = np.random.choice(list(self._trigrams[a][b].keys()), p=list(self._trigrams[a][b].values()))
        comment = ' '.join(comment)

        return self._format_comment(comment)

In [51]:
tcg = TrigramTCG()
tcg.process_comments()

In [59]:
tcg.generate_comment(random_start=True)

"! i will find fulfillment off the stage ya god damn fired! you are a americqan soldier hero.... pffff... tab anyway no need to get blown up and it gets dark. pleeezee. fatuorum o, 6 august 2007 ( utc) tolerance my ass? if it hadn't fucking ban me ill just go in the ass in the ass in the kitchen for a change- over some kiddie hackers. we can talk to you indicated that your perversions can't a cold-blooded rude, aggressive language is acceptable to paid whore."